In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.layers import Conv2D, Flatten, Dense,MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions, ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import os

import numpy as np

In [ ]:
img_height, img_width = (224,224)
batch_size = 32

In [ ]:
train_data_dir = r"split_dataset/train"
validation_data_dir = r"split_dataset/val"
test_data_dir = r"split_dataset/test"

In [ ]:
num_classes = len([folder for folder in os.listdir(train_data_dir) if os.path.isdir(os.path.join(train_data_dir, folder))])
num_classes

In [ ]:
# Data Augmentation & Preprocessing
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  
)

In [ ]:
# Train generator
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

In [ ]:
# Validation generator
validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

In [ ]:
# Test generator
test_generator = train_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=1,
    class_mode='categorical',
    subset='validation',
    shuffle=False 
)

In [ ]:
x, y = next(test_generator) 
print("Test image shape:", x.shape)

In [ ]:
# Load Pretrained ResNet50 Model (without top layers)
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(img_height, img_width, 3))

# Freeze the base model layers (so they don’t get trained)
base_model.trainable = False

# Add custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(num_classes, activation="softmax")(x)

# Create final model
model = Model(inputs=base_model.input, outputs=x)


# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss="categorical_crossentropy",
              metrics=["accuracy"])



In [ ]:
# Train the model
model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=5
)

In [ ]:
model.save("resnet50_custom.keras")

In [ ]:
test_loss, test_accuracy = model.evaluate(test_generator, verbose=2)
print("Test Accuracy:", test_accuracy)

In [ ]:
loaded_model = load_model("resnet50_custom.keras")

In [ ]:
# Get true labels
true_labels = test_generator.classes

# Get class indices (to map predictions to class names)
class_labels = list(test_generator.class_indices.keys())

# Predict probabilities
predictions = loaded_model.predict(test_generator, steps=len(test_generator), verbose=1)

# Convert probabilities to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Compute confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues",
            xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()


In [ ]:
# Print classification report
print("Classification Report:\n")
print(classification_report(true_labels, predicted_labels, target_names=class_labels))